In [1]:
import json
from collections import defaultdict
from tqdm import tqdm
import numpy as np

filename = 'yelp_academic_dataset_review.json'
target_user_ids = dict()
target_item_ids = dict()
reviews = []
with open(filename, 'rt') as f:
        for line in tqdm(f):
            data = json.loads(line)

            reviews.append({
                key: data[key]
                for key in ['review_id', 'user_id', 'business_id', 'stars', "text"]
            })
            target_user_ids[data['user_id']] = True
            target_item_ids[data['business_id']] = True
print(len(reviews))
print(json.dumps(reviews[0], indent=4))


filename = 'yelp_academic_dataset_business.json'
items = list()
with open(filename, 'rt') as f:
        for line in tqdm(f):
            data = json.loads(line)
            if data['business_id'] in target_item_ids and \
                    (data["categories"] is not None and "Restaurants" in data["categories"]):
                items.append({k : v for k,v in data.items()
                              if k not in ['name', 'address', 'latitude', 'longitude',
                                           'city', 'state', 'postal_code', 'attributes', 'hours']})
print(len(items))

filename = 'yelp_academic_dataset_user.json'
users = list()
with open(filename, 'rt') as f:
        for line in tqdm(f):
            try:
                data = json.loads(line)
                if data['user_id'] in target_user_ids:
                    users.append({k : v for k,v in data.items()
                                      if k not in ['name', 'yelping_since', 'elite', 'friends']})
            except:
                pass
print(len(users))

8635403it [01:34, 91639.22it/s] 


8635403
{
    "review_id": "lWC-xP3rd6obsecCYsGZRg",
    "user_id": "ak0TdVmGKo4pwqdJSTLwWw",
    "business_id": "buF9druCkbuXLX526sGELQ",
    "stars": 4.0,
    "text": "Apparently Prides Osteria had a rough summer as evidenced by the almost empty dining room at 6:30 on a Friday night. However new blood in the kitchen seems to have revitalized the food from other customers recent visits. Waitstaff was warm but unobtrusive. By 8 pm or so when we left the bar was full and the dining room was much more lively than it had been. Perhaps Beverly residents prefer a later seating. \n\nAfter reading the mixed reviews of late I was a little tentative over our choice but luckily there was nothing to worry about in the food department. We started with the fried dough, burrata and prosciutto which were all lovely. Then although they don't offer half portions of pasta we each ordered the entree size and split them. We chose the tagliatelle bolognese and a four cheese filled pasta in a creamy sauce w

160585it [00:03, 46270.20it/s]


50763


2189455it [01:01, 35444.88it/s]

2189454


In [2]:
import pandas as pd
item_df = pd.DataFrame(items)

from collections import Counter
category_dict = Counter()

for _, row in item_df.iterrows():
    if 'categories' not in row:
        continue
    for category in row['categories'].split(', '):
        category_dict[category] += 1
        
for category in category_dict.keys():
    item_df[category] = np.zeros(len(item_df))

for ind, row in item_df.iterrows():
    if 'categories' not in row:
        continue
    for category in row['categories'].split(', '):
        item_df.at[ind, category] = 1
item_df.drop(['categories'], axis=1, inplace = True)

/tmp/ipykernel_3063/1516477187.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  item_df[category] = np.zeros(len(item_df))


In [4]:
import gc

review_df = pd.DataFrame(reviews)
del reviews

user_df = pd.DataFrame(users)
del target_user_ids, target_item_ids, items, users, category_dict
gc.collect()

for cat in review_df.columns:
    if cat != "stars":
        review_df[cat] = review_df[cat].astype("string")
        
user_df["user_id"] = user_df["user_id"].astype("string")

gc.collect()

0

In [11]:
merge_df = pd.merge(review_df, user_df, on='user_id')


for cat in merge_df.columns:
    if cat in ["text", "business_id", "user_id", "review_id"]:
        merge_df[cat] = merge_df[cat].astype("string")
    elif cat == "average_stars":
        merge_df[cat] = merge_df[cat].astype("float16")
    elif cat == "stars":
        pass
    else:
        merge_df[cat] = merge_df[cat].astype('int16')
gc.collect()

0

In [ ]:
filename = 'yelp_academic_dataset_review.json'
with open(filename, 'rt') as f:
        for line in tqdm(f):
            data = json.loads(line)
            if data["stars"] not in [1,2,3,4,5]:
                print('helli')

1954043it [00:11, 160849.91it/s]

In [14]:
import pandas as pd
import numpy as np

merge_df = pd.read_pickle("./pickled_merged_df.pkl")
business_info_df = pd.read_pickle("item_df.pkl")

In [15]:
business_info_df.info(memory_usage="deep")
business_info_df["business_id"] = business_info_df["business_id"].astype("string")
zero_cols = [ col for col, is_zero in ((business_info_df == 0).sum() == business_info_df.shape[0]).items() if is_zero ]
business_info_df.drop(zero_cols, axis=1, inplace=True)

for i, cat in zip(business_info_df.dtypes,business_info_df.columns):
    if i in [np.float64, np.float32, np.float16]:
        business_info_df[cat] = business_info_df[cat].astype("float16")
    elif i in [np.int64, np.int32, np.int16]:
        business_info_df[cat] = business_info_df[cat].astype("int16")
    else:
        business_info_df[cat] = business_info_df[cat].astype("string")
merge_df.drop(['review_id'], axis = 1, inplace = True)

business_info_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50763 entries, 0 to 50762
Columns: 730 entries, business_id to Elementary Schools
dtypes: float64(727), int64(2), object(1)
memory usage: 286.2 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50763 entries, 0 to 50762
Columns: 729 entries, business_id to Elementary Schools
dtypes: float16(726), int16(2), string(1)
memory usage: 74.3 MB


In [17]:
merge_df.head()

,review_id,business_id,stars,text,review_count,useful,funny,cool,fans,average_stars,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,lWC-xP3rd6obsecCYsGZRg,buF9druCkbuXLX526sGELQ,4,Apparently Prides Osteria had a rough summer a...,63,44,5,14,2,4.300781,...,0,0,0,0,2,3,1,1,1,5
1,fLlML7BjkR4_fJnND_hEJw,bNZ3-0rse12NKdSVqQ30xw,4,"Came with friends, split the funghi pizza and ...",63,44,5,14,2,4.300781,...,0,0,0,0,2,3,1,1,1,5
2,pRtbswupEVIG1Ykj9xkL7Q,BVsIaKL-8QXVjt0Z9WoFWw,4,Went for late lunch had the combination seafoo...,63,44,5,14,2,4.300781,...,0,0,0,0,2,3,1,1,1,5
3,fUYl6bnZy4bSGnbPAizXug,4MClvr12OXBNvGu8h1yGpA,5,"We were super excited to try Sarma, having bee...",63,44,5,14,2,4.300781,...,0,0,0,0,2,3,1,1,1,5
4,jHh2LIXNsnJCMUiyI9pt5w,2vH58mhkEl8GdcDug1OwWg,5,So glad we made the trip to Woburn for Gene's ...,63,44,5,14,2,4.300781,...,0,0,0,0,2,3,1,1,1,5


In [ ]:
merge_df = pd.merge(merge_df, business_info_df, on='business_id')
merge_df.drop(['business_id'], axis=1, inplace = True)

NameError: name 'merge_df' is not defined

In [ ]:
business_info_df.head(1)

In [ ]:
merge_df.head(1)

In [38]:
#https://www.kaggle.com/subhambanerjee1606/aspect-opinion-extraction-and-modelling#Spacy-Dependency-Parsing
    
#https://www.kaggle.com/courierliu/yelp-feature-base-recommendation

In [14]:
    print(i, type(i), i == np.object_)
    break

object <class 'numpy.dtype[object_]'> True


In [15]:
np.float64

numpy.float64

In [19]:
np.string_

numpy.bytes_